In [65]:
! pip install gradio_client
! pip install groq
! pip install transformers

/home/tjyana/.pyenv/versions/3.10.6/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [66]:
import itertools
import re
import numpy as np
import pandas as pd
# import openai
import pandas as pd
import itertools
# import streamlit as st
import pickle
# Anna's imports
from transformers import FlaxAutoModelForSeq2SeqLM
from transformers import AutoTokenizer
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from gradio_client import Client

from groq import Groq
import config

df = pd.read_parquet('Halved-DF.parquet.gzip')

def get_model():
    with open("model.pickle", "rb") as f:
        model = pickle.load(f)
    return model
model = get_model()

# combinations_of_two -> DONE
-> working correctly (assuming desired output format is: list of tuples, containing all possible combinations of two ingredients)

In [67]:
# # Updated 5/21/2024 by TJ

# def combinations_of_two(ingredients_input): ###dealt with the issue of missing space crash

#     '''
#     The function generates all unique pairs of ingredients that can be made from the input list of ingredients.
#     NOTE FOR FRONT-END: The output of this function is the input for data_query()
    
#     UPDATES
#     5/22/2024 by TJ:
#     - Modified it quite a bit so output format is one list of tuples
#     - Removed the powerset stuff
    
#     Inputs (1): 
#     ingredients_input (from: user input in app) =  single string with ingredients separated by commas and a space
    
#     Outputs (1): 
#     ingredients_combinations (to: data_query) = a list of tuples, containing all possible combinations of 2 (so if it's 4 ingredients, a list of 6 tuples and nothing more)

#     '''

#     # split into list of individual ingredients
#     ingredients_list = []
#     ingredients = re.split('\s|,', ingredients_input) 
#     [ingredients_list.append(ingredient) for ingredient in ingredients if ingredient != '']
    
#     # create list of all ingredient combinations
#     ingredients_combinations = []
#     combinations = itertools.combinations(ingredients_list, 2)
#     ingredients_combinations = list(combinations)

#     return ingredients_combinations

In [68]:
def combinations_of_two(ingredients_input): ###dealt with the issue of missing space crash

    '''
    The function generates all unique pairs of ingredients that can be made from the input list of ingredients.
    NOTE FOR FRONT-END: The output of this function is the input for data_query()
    
    Inputs (1): 
    ingredients_input (from: user input in app) =  single string with ingredients separated by commas and a space
    
    Outputs (1): 
    ingredients_combinations (to: data_query) = a list of tuples, containing all possible combinations of 2 (so if it's 4 ingredients, a list of 6 tuples and nothing more)

    '''

    ingredients_combinations = []
    powerpowerset = []
    ingredients = re.split(r',', ingredients_input.strip())
    ingredients_list = list(set(ingredient.strip() for ingredient in ingredients))
    for r in range(len(ingredients_list)+1):
        combinations = itertools.combinations(ingredients_list, r)
        #powerset.extend(subset for subset in combinations if len(subset) > 1)
        for comb in combinations:
            if len(comb) > 1:
                if len(comb) < 3:
                    ingredients_combinations.append(comb)
                else:
                    powerpowerset.append(comb)
                    for power in powerpowerset:
                        lowerset = []
                        combins = itertools.combinations(power, 2)
                        for arrange in combins:
                            lowerset.append(arrange)
                    ingredients_combinations.append(lowerset)
    return ingredients_combinations



### Testing combinations_of_two with tuples only
4 ingredients -> 6 tuples
<br>5 ingredients -> 10 tuples
<br>6 ingredients -> 15 tuples

In [69]:
# 4 ingredients -> 1 list of 6 unique tuples
ingredients_input = 'chicken, garlic, salt, tomato'

ingredients_combinations = combinations_of_two(ingredients_input)
print(len(ingredients_combinations))
print(ingredients_combinations)

11
[('tomato', 'salt'), ('tomato', 'garlic'), ('tomato', 'chicken'), ('salt', 'garlic'), ('salt', 'chicken'), ('garlic', 'chicken'), [('tomato', 'salt'), ('tomato', 'garlic'), ('salt', 'garlic')], [('tomato', 'salt'), ('tomato', 'chicken'), ('salt', 'chicken')], [('tomato', 'garlic'), ('tomato', 'chicken'), ('garlic', 'chicken')], [('salt', 'garlic'), ('salt', 'chicken'), ('garlic', 'chicken')], [('tomato', 'salt'), ('tomato', 'garlic'), ('tomato', 'chicken'), ('salt', 'garlic'), ('salt', 'chicken'), ('garlic', 'chicken')]]


In [70]:
# 5 ingredients -> 1 list of 10 unique tuples
ingredients_input = 'chicken, garlic, salt, tomato, cheese'

ingredients_combinations = combinations_of_two(ingredients_input)
print(len(ingredients_combinations))
print(ingredients_combinations)

26
[('salt', 'chicken'), ('salt', 'tomato'), ('salt', 'garlic'), ('salt', 'cheese'), ('chicken', 'tomato'), ('chicken', 'garlic'), ('chicken', 'cheese'), ('tomato', 'garlic'), ('tomato', 'cheese'), ('garlic', 'cheese'), [('salt', 'chicken'), ('salt', 'tomato'), ('chicken', 'tomato')], [('salt', 'chicken'), ('salt', 'garlic'), ('chicken', 'garlic')], [('salt', 'chicken'), ('salt', 'cheese'), ('chicken', 'cheese')], [('salt', 'tomato'), ('salt', 'garlic'), ('tomato', 'garlic')], [('salt', 'tomato'), ('salt', 'cheese'), ('tomato', 'cheese')], [('salt', 'garlic'), ('salt', 'cheese'), ('garlic', 'cheese')], [('chicken', 'tomato'), ('chicken', 'garlic'), ('tomato', 'garlic')], [('chicken', 'tomato'), ('chicken', 'cheese'), ('tomato', 'cheese')], [('chicken', 'garlic'), ('chicken', 'cheese'), ('garlic', 'cheese')], [('tomato', 'garlic'), ('tomato', 'cheese'), ('garlic', 'cheese')], [('salt', 'chicken'), ('salt', 'tomato'), ('salt', 'garlic'), ('chicken', 'tomato'), ('chicken', 'garlic'), ('to

In [71]:
# 6 ingredients -> 1 list of 15 unique tuples
ingredients_input = 'tomato, beef, onion, salt, garlic, pepper'

ingredients_combinations = combinations_of_two(ingredients_input)
print(len(ingredients_combinations))
print(ingredients_combinations)

57
[('beef', 'onion'), ('beef', 'salt'), ('beef', 'tomato'), ('beef', 'garlic'), ('beef', 'pepper'), ('onion', 'salt'), ('onion', 'tomato'), ('onion', 'garlic'), ('onion', 'pepper'), ('salt', 'tomato'), ('salt', 'garlic'), ('salt', 'pepper'), ('tomato', 'garlic'), ('tomato', 'pepper'), ('garlic', 'pepper'), [('beef', 'onion'), ('beef', 'salt'), ('onion', 'salt')], [('beef', 'onion'), ('beef', 'tomato'), ('onion', 'tomato')], [('beef', 'onion'), ('beef', 'garlic'), ('onion', 'garlic')], [('beef', 'onion'), ('beef', 'pepper'), ('onion', 'pepper')], [('beef', 'salt'), ('beef', 'tomato'), ('salt', 'tomato')], [('beef', 'salt'), ('beef', 'garlic'), ('salt', 'garlic')], [('beef', 'salt'), ('beef', 'pepper'), ('salt', 'pepper')], [('beef', 'tomato'), ('beef', 'garlic'), ('tomato', 'garlic')], [('beef', 'tomato'), ('beef', 'pepper'), ('tomato', 'pepper')], [('beef', 'garlic'), ('beef', 'pepper'), ('garlic', 'pepper')], [('onion', 'salt'), ('onion', 'tomato'), ('salt', 'tomato')], [('onion', 's

In [72]:
# missing spaces and commas
ingredients_input = 'tomato,beef,  onion chicken, apple'

ingredients_combinations = combinations_of_two(ingredients_input)
print(len(ingredients_combinations))
print(ingredients_combinations)

11
[('tomato', 'beef'), ('tomato', 'onion chicken'), ('tomato', 'apple'), ('beef', 'onion chicken'), ('beef', 'apple'), ('onion chicken', 'apple'), [('tomato', 'beef'), ('tomato', 'onion chicken'), ('beef', 'onion chicken')], [('tomato', 'beef'), ('tomato', 'apple'), ('beef', 'apple')], [('tomato', 'onion chicken'), ('tomato', 'apple'), ('onion chicken', 'apple')], [('beef', 'onion chicken'), ('beef', 'apple'), ('onion chicken', 'apple')], [('tomato', 'beef'), ('tomato', 'onion chicken'), ('tomato', 'apple'), ('beef', 'onion chicken'), ('beef', 'apple'), ('onion chicken', 'apple')]]


In [73]:
# ingredients with higher compatibility

ingredients_input = 'chicken, onion, garlic, cheese'
ingredients_combinations = combinations_of_two(ingredients_input)
print(len(ingredients_combinations))
print(ingredients_combinations)

11
[('onion', 'cheese'), ('onion', 'garlic'), ('onion', 'chicken'), ('cheese', 'garlic'), ('cheese', 'chicken'), ('garlic', 'chicken'), [('onion', 'cheese'), ('onion', 'garlic'), ('cheese', 'garlic')], [('onion', 'cheese'), ('onion', 'chicken'), ('cheese', 'chicken')], [('onion', 'garlic'), ('onion', 'chicken'), ('garlic', 'chicken')], [('cheese', 'garlic'), ('cheese', 'chicken'), ('garlic', 'chicken')], [('onion', 'cheese'), ('onion', 'garlic'), ('onion', 'chicken'), ('cheese', 'garlic'), ('cheese', 'chicken'), ('garlic', 'chicken')]]


### Testing combinations_of_two with the mixed output

In [74]:
# 4 ingredients

ingredients_input = 'chicken, onion, garlic, cheese'

ingredients_combinations = combinations_of_two(ingredients_input)
ingredients_combinations

[('onion', 'cheese'),
 ('onion', 'garlic'),
 ('onion', 'chicken'),
 ('cheese', 'garlic'),
 ('cheese', 'chicken'),
 ('garlic', 'chicken'),
 [('onion', 'cheese'), ('onion', 'garlic'), ('cheese', 'garlic')],
 [('onion', 'cheese'), ('onion', 'chicken'), ('cheese', 'chicken')],
 [('onion', 'garlic'), ('onion', 'chicken'), ('garlic', 'chicken')],
 [('cheese', 'garlic'), ('cheese', 'chicken'), ('garlic', 'chicken')],
 [('onion', 'cheese'),
  ('onion', 'garlic'),
  ('onion', 'chicken'),
  ('cheese', 'garlic'),
  ('cheese', 'chicken'),
  ('garlic', 'chicken')]]

In [75]:
# 5 ingredients

ingredients_input = 'chicken, onion, garlic, cheese, tomato'

ingredients_combinations = combinations_of_two(ingredients_input)
ingredients_combinations

[('onion', 'chicken'),
 ('onion', 'tomato'),
 ('onion', 'garlic'),
 ('onion', 'cheese'),
 ('chicken', 'tomato'),
 ('chicken', 'garlic'),
 ('chicken', 'cheese'),
 ('tomato', 'garlic'),
 ('tomato', 'cheese'),
 ('garlic', 'cheese'),
 [('onion', 'chicken'), ('onion', 'tomato'), ('chicken', 'tomato')],
 [('onion', 'chicken'), ('onion', 'garlic'), ('chicken', 'garlic')],
 [('onion', 'chicken'), ('onion', 'cheese'), ('chicken', 'cheese')],
 [('onion', 'tomato'), ('onion', 'garlic'), ('tomato', 'garlic')],
 [('onion', 'tomato'), ('onion', 'cheese'), ('tomato', 'cheese')],
 [('onion', 'garlic'), ('onion', 'cheese'), ('garlic', 'cheese')],
 [('chicken', 'tomato'), ('chicken', 'garlic'), ('tomato', 'garlic')],
 [('chicken', 'tomato'), ('chicken', 'cheese'), ('tomato', 'cheese')],
 [('chicken', 'garlic'), ('chicken', 'cheese'), ('garlic', 'cheese')],
 [('tomato', 'garlic'), ('tomato', 'cheese'), ('garlic', 'cheese')],
 [('onion', 'chicken'),
  ('onion', 'tomato'),
  ('onion', 'garlic'),
  ('chicke

In [76]:
# 6 ingredients

ingredients_input = 'chicken, onion, garlic, cheese, tomato, salt'

ingredients_combinations = combinations_of_two(ingredients_input)
ingredients_combinations

[('onion', 'salt'),
 ('onion', 'chicken'),
 ('onion', 'tomato'),
 ('onion', 'garlic'),
 ('onion', 'cheese'),
 ('salt', 'chicken'),
 ('salt', 'tomato'),
 ('salt', 'garlic'),
 ('salt', 'cheese'),
 ('chicken', 'tomato'),
 ('chicken', 'garlic'),
 ('chicken', 'cheese'),
 ('tomato', 'garlic'),
 ('tomato', 'cheese'),
 ('garlic', 'cheese'),
 [('onion', 'salt'), ('onion', 'chicken'), ('salt', 'chicken')],
 [('onion', 'salt'), ('onion', 'tomato'), ('salt', 'tomato')],
 [('onion', 'salt'), ('onion', 'garlic'), ('salt', 'garlic')],
 [('onion', 'salt'), ('onion', 'cheese'), ('salt', 'cheese')],
 [('onion', 'chicken'), ('onion', 'tomato'), ('chicken', 'tomato')],
 [('onion', 'chicken'), ('onion', 'garlic'), ('chicken', 'garlic')],
 [('onion', 'chicken'), ('onion', 'cheese'), ('chicken', 'cheese')],
 [('onion', 'tomato'), ('onion', 'garlic'), ('tomato', 'garlic')],
 [('onion', 'tomato'), ('onion', 'cheese'), ('tomato', 'cheese')],
 [('onion', 'garlic'), ('onion', 'cheese'), ('garlic', 'cheese')],
 [(

# data_query
Issues:
- output is not including all the combinations for some inputs
- higher compatibility lists seem to return more rows? not sure

In [77]:
def data_query(ingredients_combinations): ##Added a penalty of -5 for pairings that are not in the dataframe
    '''
    Scores all combinations of two ingredients.
    
    Inputs (1): 
    ingredients_combinations (from: combinations_of_two) = a list of tuples, containing all possible combinations of 2 as  
    
    Outputs (1): 
    df_comb (to: muse_comb) = a datafrome with columns 'Combination' and 'Score', containing ingredient combinations and their scores as a dataframe  (for 4 ingredients: all 6 combinations and their respective scores)
    
    '''
    
    data = []
    for combination in ingredients_combinations:
        if len(combination) < 3:
            ingredient1, ingredient2 = combination
            query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
            score = df.query(query_str)['scaled_col'].values
            if len(score) > 0:
                data.append({'Combination': combination, 'Score': score})
            else:
                continue
        else:
            scores = []
            for i in combination:
                ingredient1, ingredient2 = i
                query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
                score = df.query(query_str)['scaled_col'].values
                if len(score) > 0:
                    scores.append(score[0])
                else:
                    scores.append(-5)
            data.append({'Combination': combination, 'Score': scores})
    df_comb = pd.DataFrame(data)
    return df_comb

In [78]:
# # # trying to understand / re-create
# def data_query(ingredients_combinations): ##Added a penalty of -5 for pairings that are not in the dataframe
#     '''
#     Scores all combinations of two ingredients.
    
#     Inputs (1): 
#     ingredients_combinations (from: combinations_of_two) = a list of tuples, containing all possible combinations of 2 as  
    
#     Outputs (1): 
#     df_comb (to: muse_comb) = a datafrome with columns 'Combination' and 'Score', containing ingredient combinations and their scores as a dataframe  (for 4 ingredients: all 6 combinations and their respective scores)
    
#     '''
    
#     data = []
#     for combination in ingredients_combinations:
#         if len(combination) < 3:
#             ingredient1, ingredient2 = combination
#             query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
#             score = df.query(query_str)['scaled_col'].values
#             print(score)
#             if len(score) > 0:
#                 data.append({'Combination': combination, 'Score': score})
#             else:
#                 continue
#         else:
#             scores = []
#             for i in combination:
#                 ingredient1, ingredient2 = i
#                 query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
#                 score = df.query(query_str)['scaled_col'].values
#                 if len(score) > 0:
#                     scores.append(score[0])
#                 else:
#                     scores.append(-5)
#             data.append({'Combination': combination, 'Score': scores})
#     print(data)

#     df_comb = pd.DataFrame(data)
#     return df_comb

### Testing data_query with tuples

In [79]:
# 6 tuples -> should return 6 rows

ingredients_combinations = [('chicken', 'garlic'),
 ('chicken', 'salt'),
 ('chicken', 'tomato'),
 ('garlic', 'salt'),
 ('garlic', 'tomato'),
 ('salt', 'tomato')]

df_comb = data_query(ingredients_combinations)
df_comb

,Combination,Score
0,"(chicken, garlic)",[1.1800811339314616]
1,"(chicken, salt)",[1.4353972677154387]
2,"(garlic, salt)",[1.5533159218950603]


In [80]:
# 15 tuples -> should return 15 rows

ingredients_combinations = [('tomato', 'beef'),
 ('tomato', 'onion'),
 ('tomato', 'salt'),
 ('tomato', 'garlic'),
 ('tomato', 'pepper'),
 ('beef', 'onion'),
 ('beef', 'salt'),
 ('beef', 'garlic'),
 ('beef', 'pepper'),
 ('onion', 'salt'),
 ('onion', 'garlic'),
 ('onion', 'pepper'),
 ('salt', 'garlic'),
 ('salt', 'pepper'),
 ('garlic', 'pepper')]

df_comb = data_query(ingredients_combinations)
df_comb

,Combination,Score
0,"(beef, onion)",[1.3407460751179292]
1,"(beef, salt)",[1.3667368322711522]
2,"(beef, garlic)",[1.1385957417039734]
3,"(beef, pepper)",[1.1819136702926785]
4,"(onion, salt)",[2.0670735182650013]
5,"(onion, garlic)",[1.4235409132992736]
6,"(onion, pepper)",[1.4949048817870405]
7,"(salt, garlic)",[1.5533159218950603]
8,"(salt, pepper)",[3.3564504877818235]
9,"(garlic, pepper)",[1.1902734549399014]


In [81]:
# 6 tuples -> should return 6 rows
# high compatiblity

ingredients_combinations = [('chicken', 'onion'), ('chicken', 'garlic'), ('chicken', 'cheese'), ('onion', 'garlic'), ('onion', 'cheese'), ('garlic', 'cheese')]

df_comb = data_query(ingredients_combinations)
df_comb

,Combination,Score
0,"(chicken, onion)",[1.3248796627107513]
1,"(chicken, garlic)",[1.1800811339314616]
2,"(chicken, cheese)",[1.0177501859513405]
3,"(onion, garlic)",[1.4235409132992736]
4,"(onion, cheese)",[1.4120038875676597]
5,"(garlic, cheese)",[1.0944496012124034]


### Testing data_query with the mixed output

In [82]:
# 4 ingredients

ingredients_input = 'chicken, onion, garlic, cheese'

ingredients_combinations = combinations_of_two(ingredients_input)
df_comb = data_query(ingredients_combinations)
df_comb

,Combination,Score
0,"(onion, cheese)",[1.4120038875676597]
1,"(onion, garlic)",[1.4235409132992736]
2,"(onion, chicken)",[1.3248796627107513]
3,"(cheese, garlic)",[1.0944496012124034]
4,"(cheese, chicken)",[1.0177501859513405]
5,"(garlic, chicken)",[1.1800811339314616]
6,"[(onion, cheese), (onion, garlic), (cheese, ga...","[1.4120038875676597, 1.4235409132992736, 1.094..."
7,"[(onion, cheese), (onion, chicken), (cheese, c...","[1.4120038875676597, 1.3248796627107513, 1.017..."
8,"[(onion, garlic), (onion, chicken), (garlic, c...","[1.4235409132992736, 1.3248796627107513, 1.180..."
9,"[(cheese, garlic), (cheese, chicken), (garlic,...","[1.0944496012124034, 1.0177501859513405, 1.180..."


In [83]:
# 5 ingredients

ingredients_input = 'chicken, onion, garlic, cheese, tomato'

ingredients_combinations = combinations_of_two(ingredients_input)
df_comb = data_query(ingredients_combinations)
df_comb

,Combination,Score
0,"(onion, chicken)",[1.3248796627107513]
1,"(onion, garlic)",[1.4235409132992736]
2,"(onion, cheese)",[1.4120038875676597]
3,"(chicken, garlic)",[1.1800811339314616]
4,"(chicken, cheese)",[1.0177501859513405]
5,"(garlic, cheese)",[1.0944496012124034]
6,"[(onion, chicken), (onion, tomato), (chicken, ...","[1.3248796627107513, -5, -5]"
7,"[(onion, chicken), (onion, garlic), (chicken, ...","[1.3248796627107513, 1.4235409132992736, 1.180..."
8,"[(onion, chicken), (onion, cheese), (chicken, ...","[1.3248796627107513, 1.4120038875676597, 1.017..."
9,"[(onion, tomato), (onion, garlic), (tomato, ga...","[-5, 1.4235409132992736, -5]"


In [84]:
# 6 ingredients

ingredients_input = 'chicken, onion, garlic, cheese, tomato, salt'

ingredients_combinations = combinations_of_two(ingredients_input)
df_comb = data_query(ingredients_combinations)
df_comb

,Combination,Score
0,"(onion, salt)",[2.0670735182650013]
1,"(onion, chicken)",[1.3248796627107513]
2,"(onion, garlic)",[1.4235409132992736]
3,"(onion, cheese)",[1.4120038875676597]
4,"(salt, chicken)",[1.4353972677154387]
5,"(salt, garlic)",[1.5533159218950603]
6,"(salt, cheese)",[1.3703804244076925]
7,"(chicken, garlic)",[1.1800811339314616]
8,"(chicken, cheese)",[1.0177501859513405]
9,"(garlic, cheese)",[1.0944496012124034]


# muse_comb
- output is in desired format (a list of strings) so code will run! However...

<br>Issues:
<br><s>- the output format of data_query needs to be confirmed first for proper testing. is it ok that combinations are in a tuple and score is in a list? (maybe do muse_comb later)</s>
- pretty slow for 5+ ingredients
- 3 ingredients: 2 of the ingredients are just single letters and not ingredients

In [85]:
def muse_comb(df_comb): ###If this takes too long, consider taking the nested calculate_sum(array) outside of the function
    '''
     The function calculates the sum of the "Score" values and returns the three combinations with the largest sums

     NOTE FOR FRONT-END: The return is a list of lists so access the values by indexing e.g. output[0]

                         The output of this function is the input for the recipe generator

                         We might need a function to convert each lists into strings if
                         the recipe generator doesn't do this automatically.
                         
    Inputs (1): 
    df_comb (from: data_query) = a datafrome with columns 'Combination' and 'Score', containing ingredient combinations and their scores as a dataframe  (for 4 ingredients: all 6 combinations and their respective scores)
    
    Output (1):
    ingredients_list (to: recipe_generator) = a list of 3 lists, containing the 3 ingredients combinations with highest scores
    [['yeast', 'butter', 'eggs', 'pepper', 'cabbage', 'pork', 'flour', 'sugar'],
    ['butter', 'eggs', 'pepper', 'cabbage', 'pork', 'flour', 'sugar'],
    ['yeast', 'butter', 'eggs', 'pepper', 'cabbage', 'flour', 'sugar']]
    
    '''

    def calculate_sum(array):
        return sum(array)

    def ingredients_to_lists(lists):
        ingredients_list = []
        for i in range(3):
            tmp_list = []
            for x in lists[i]:
                tmp_list.append(x[0])
                tmp_list.append(x[1])
            ingredients_list.append(list(set(tmp_list)))

        return ingredients_list

    for i in range(len(df_comb)):
        df_comb["Sum"] = df_comb["Score"].apply(calculate_sum)

    max_values = df_comb.nlargest(3, "Sum")

    max_values = max_values["Combination"].reset_index(drop=True)

    ingredients_lists = ingredients_to_lists(max_values)

    return ingredients_lists

### Testing muse_comb with pattern 1
(might need to fix data_query output first)

In [86]:
# 6 tuples

ingredients_combinations = [('chicken', 'garlic'),
 ('chicken', 'salt'),
 ('chicken', 'tomato'),
 ('garlic', 'salt'),
 ('garlic', 'tomato'),
 ('salt', 'tomato')]

df_comb = data_query(ingredients_combinations)

ingredients_lists = muse_comb(df_comb)
ingredients_lists

[['a', 's', 'g'], ['h', 'c', 'a', 's'], ['h', 'c', 'a', 'g']]

In [87]:
# 15 tuples

ingredients_combinations = [('tomato', 'beef'),
 ('tomato', 'onion'),
 ('tomato', 'salt'),
 ('tomato', 'garlic'),
 ('tomato', 'pepper'),
 ('beef', 'onion'),
 ('beef', 'salt'),
 ('beef', 'garlic'),
 ('beef', 'pepper'),
 ('onion', 'salt'),
 ('onion', 'garlic'),
 ('onion', 'pepper'),
 ('salt', 'garlic'),
 ('salt', 'pepper'),
 ('garlic', 'pepper')]

df_comb = data_query(ingredients_combinations)

ingredients_lists = muse_comb(df_comb)
ingredients_lists

[['e', 'p', 'a', 's'], ['n', 'a', 's', 'o'], ['a', 's', 'g']]

### Testing muse_comb with pattern 2

In [88]:
# 3 ingredients

ingredients_input = 'chicken, onion, garlic'

ingredients_combinations = combinations_of_two(ingredients_input)
df_comb = data_query(ingredients_combinations)
muse_comb(df_comb)

[['onion', 'garlic', 'chicken'], ['n', 'a', 'o', 'g'], ['n', 'h', 'c', 'o']]

In [89]:
# 4 ingredients

ingredients_input = 'chicken, onion, garlic, cheese'

ingredients_combinations = combinations_of_two(ingredients_input)
df_comb = data_query(ingredients_combinations)
muse_comb(df_comb)

[['onion', 'cheese', 'garlic', 'chicken'],
 ['onion', 'cheese', 'garlic'],
 ['onion', 'garlic', 'chicken']]

In [90]:
# 5 ingredients

ingredients_input = 'chicken, onion, garlic, cheese, tomato'

ingredients_combinations = combinations_of_two(ingredients_input)
df_comb = data_query(ingredients_combinations)
muse_comb(df_comb)

[['onion', 'cheese', 'garlic', 'chicken'],
 ['onion', 'cheese', 'garlic'],
 ['onion', 'garlic', 'chicken']]

In [91]:
# 6 ingredients

ingredients_input = 'chicken, onion, garlic, cheese, tomato, salt'

ingredients_combinations = combinations_of_two(ingredients_input)
df_comb = data_query(ingredients_combinations)
muse_comb(df_comb)

[['onion', 'salt', 'chicken', 'garlic', 'cheese'],
 ['onion', 'salt', 'garlic', 'chicken'],
 ['onion', 'salt', 'garlic', 'cheese']]

# recipe_generator -> DONE

Working correctly! Only issue is client-side seems to go down every once in a while (400 error).

In [92]:
def recipe_generator(ingredients_lists):

    '''
    Takes ingredients_list from muse_comb and returns the actual recipes with titles, ingredients, and directions.

    Updates:
    5/22/2024 by TJ:
    - Added config.py file to protect API Key. 

    Inputs (1):
    ingredients_list (from: muse_comb) = a list of 3 lists, containing the 3 ingredients combinations with highest score

    Outputs (1):
    recipe_list (to: get_scores, final_recipe) = a list of 3 dictionaries with 3 keys each: 'title', 'ingredients', 'directions', containing info for the 3 recipes

    '''


    api_key = config.api_key2
    client = Groq(
    api_key=api_key
    )
    recipe_list = []

    if len(ingredients_lists) == 1:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"Suggest one recipe with {ingredients_lists} only. The final format of the output should contain Title, Ingredients and Directions only",
                }
            ],
            model="llama3-8b-8192",
        )

        recipe = chat_completion.choices[0].message.content

        parts = recipe.split("**")
        title = parts[1].strip()
        ingredients = parts[4].strip()
        directions = parts[6].strip()

        recipe_dict = {}
        recipe_dict['title'] = title
        recipe_dict['ingredients'] = ingredients
        recipe_dict['directions'] = directions

        recipe_list.append(recipe_dict)

    else:
      for i in range(len(ingredients_lists)):
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"Suggest one recipe with {ingredients_lists[i]} only. The final format of the output should contain Title, Ingredients and Directions only",
                }
            ],
            model="llama3-8b-8192",
        )

        recipe = chat_completion.choices[0].message.content

        parts = recipe.split("**")
        title = parts[1].strip()
        ingredients = parts[4].strip()
        directions = parts[6].strip()

        recipe_dict = {}
        recipe_dict['title'] = title
        recipe_dict['ingredients'] = ingredients
        recipe_dict['directions'] = directions

        recipe_list.append(recipe_dict)

    return recipe_list

### Testing recipe_generator

In [93]:
ingredients_lists = [['tomato', 'chicken', 'garlic'], ['chicken', 'onion', 'salt'], ['onion', 'salt', 'garlic']]

recipe_list = recipe_generator(ingredients_lists)
recipe_list

[{'title': 'Roasted Chicken with Tomato and Garlic',
  'ingredients': '* 1 1/2 pounds chicken breast or thighs\n* 2 large tomatoes, sliced\n* 3 cloves garlic, peeled and minced',
  'directions': '1. Preheat oven to 425°F (220°C).\n2. In a large bowl, toss chicken with garlic, tomato slices, and a pinch of salt.\n3. Spread on a large baking sheet lined with parchment paper and roast for 35-40 minutes or until cooked through.'},
 {'title': 'Grilled Chicken with Onion',
  'ingredients': '* 1 piece of chicken\n* 1/2 onion\n* salt to taste',
  'directions': '1. Preheat grill to medium-high heat.\n2. Season chicken with salt.\n3. Grill chicken for 5-6 minutes per side or until cooked through.\n4. Meanwhile, slice the onion into rings.\n5. Grill the onion rings for 2-3 minutes per side or until caramelized.\n6. Serve chicken with grilled onion rings.'},
 {'title': 'Garlic Onion Delight',
  'ingredients': '* 2 onions, thinly sliced\n* 1/2 teaspoon salt\n* 2 cloves garlic, minced',
  'direction

### Testing recipe_generator all the way through

In [94]:
# 3 ingredients

ingredients_input = 'chicken, onion, garlic'

ingredients_combinations = combinations_of_two(ingredients_input)
print('ingredients_combinations: \n', ingredients_combinations)
df_comb = data_query(ingredients_combinations)
print('\n df_comb: \n', df_comb)
ingredients_list = muse_comb(df_comb)
print('\n ingredients_list: \n', ingredients_list)
recipe_list = recipe_generator(ingredients_list)
recipe_list

ingredients_combinations: 
 [('onion', 'garlic'), ('onion', 'chicken'), ('garlic', 'chicken'), [('onion', 'garlic'), ('onion', 'chicken'), ('garlic', 'chicken')]]

 df_comb: 
                                          Combination  \
0                                    (onion, garlic)   
1                                   (onion, chicken)   
2                                  (garlic, chicken)   
3  [(onion, garlic), (onion, chicken), (garlic, c...   

                                               Score  
0                               [1.4235409132992736]  
1                               [1.3248796627107513]  
2                               [1.1800811339314616]  
3  [1.4235409132992736, 1.3248796627107513, 1.180...  

 ingredients_list: 
 [['onion', 'garlic', 'chicken'], ['n', 'a', 'o', 'g'], ['n', 'h', 'c', 'o']]


[{'title': 'Chicken and Onion Stir-Fry',
  'ingredients': '* 1 lb chicken breast or thighs, cut into bite-sized pieces\n* 1 medium onion, diced\n* 3 cloves garlic, minced',
  'directions': '1. Heat a large skillet or wok over medium-high heat.\n2. Add the chicken to the skillet and cook until browned, about 5-7 minutes. Remove from the skillet and set aside.\n3. In the same skillet, add the diced onion and cook until softened, about 3-4 minutes.\n4. Add the minced garlic to the skillet and cook for an additional 1 minute, until fragrant.\n5. Add the cooked chicken back into the skillet and stir to combine with the onion and garlic mixture.\n6. Cook for an additional 1-2 minutes, until the chicken is coated in the onion and garlic mixture.\n7. Serve hot and enjoy!'},
 {'title': 'Noangle',
  'ingredients': '* Nanograms of Onions\n* Angular Aloes',
  'directions': '1. Ogre Around Noon.\n2. Anally Adjust Onions.\n3. Naive Observation Generate Angularity.'},
 {'title': 'Title:',
  'ingredie

In [95]:
# 4 ingredients

ingredients_input = 'chicken, onion, garlic, cheese'

ingredients_combinations = combinations_of_two(ingredients_input)
print('ingredients_combinations: \n', ingredients_combinations)
df_comb = data_query(ingredients_combinations)
print('\n df_comb: \n', df_comb)
ingredients_list = muse_comb(df_comb)
print('\n ingredients_list: \n', ingredients_list)
recipe_list = recipe_generator(ingredients_list)
recipe_list

ingredients_combinations: 
 [('onion', 'cheese'), ('onion', 'garlic'), ('onion', 'chicken'), ('cheese', 'garlic'), ('cheese', 'chicken'), ('garlic', 'chicken'), [('onion', 'cheese'), ('onion', 'garlic'), ('cheese', 'garlic')], [('onion', 'cheese'), ('onion', 'chicken'), ('cheese', 'chicken')], [('onion', 'garlic'), ('onion', 'chicken'), ('garlic', 'chicken')], [('cheese', 'garlic'), ('cheese', 'chicken'), ('garlic', 'chicken')], [('onion', 'cheese'), ('onion', 'garlic'), ('onion', 'chicken'), ('cheese', 'garlic'), ('cheese', 'chicken'), ('garlic', 'chicken')]]

 df_comb: 
                                           Combination  \
0                                     (onion, cheese)   
1                                     (onion, garlic)   
2                                    (onion, chicken)   
3                                    (cheese, garlic)   
4                                   (cheese, chicken)   
5                                   (garlic, chicken)   
6   [(onion, cheese),

[{'title': 'Garlic Chicken Fritters',
  'ingredients': '* 1 onion, finely chopped\n* 1 cup shredded cheese\n* 2 cloves garlic, minced\n* 1 pound cooked chicken, shredded',
  'directions': '1. In a large bowl, combine chopped onion, shredded cheese, minced garlic, and shredded chicken.\n2. Mix well until all the ingredients are well combined.\n3. Using your hands, shape the mixture into small patties.\n4. Heat a non-stick pan with a small amount of oil.\n5. Add the patties and cook until golden brown and crispy.\n6. Serve hot and enjoy!'},
 {'title': 'Garlic and Cheese Fritter',
  'ingredients': '- 1 onion, finely chopped\n- 1/2 cup grated cheese\n- 3 cloves of garlic, minced',
  'directions': '1. Heat a pan with a non-stick surface over medium heat.\n2. Add the chopped onion and sauté until translucent.\n3. Add the minced garlic and cook for an additional minute.\n4. Sprinkle the grated cheese and stir until melted and combined with the onion mixture.\n5. Serve the fritter garnished wi

In [96]:
# 5 ingredients

ingredients_input = 'chicken, onion, garlic, cheese, tomato'

ingredients_combinations = combinations_of_two(ingredients_input)
print('ingredients_combinations: \n', ingredients_combinations)
df_comb = data_query(ingredients_combinations)
print('\n df_comb: \n', df_comb)
ingredients_list = muse_comb(df_comb)
print('\n ingredients_list: \n', ingredients_list)
recipe_list = recipe_generator(ingredients_list)
recipe_list

ingredients_combinations: 
 [('onion', 'chicken'), ('onion', 'tomato'), ('onion', 'garlic'), ('onion', 'cheese'), ('chicken', 'tomato'), ('chicken', 'garlic'), ('chicken', 'cheese'), ('tomato', 'garlic'), ('tomato', 'cheese'), ('garlic', 'cheese'), [('onion', 'chicken'), ('onion', 'tomato'), ('chicken', 'tomato')], [('onion', 'chicken'), ('onion', 'garlic'), ('chicken', 'garlic')], [('onion', 'chicken'), ('onion', 'cheese'), ('chicken', 'cheese')], [('onion', 'tomato'), ('onion', 'garlic'), ('tomato', 'garlic')], [('onion', 'tomato'), ('onion', 'cheese'), ('tomato', 'cheese')], [('onion', 'garlic'), ('onion', 'cheese'), ('garlic', 'cheese')], [('chicken', 'tomato'), ('chicken', 'garlic'), ('tomato', 'garlic')], [('chicken', 'tomato'), ('chicken', 'cheese'), ('tomato', 'cheese')], [('chicken', 'garlic'), ('chicken', 'cheese'), ('garlic', 'cheese')], [('tomato', 'garlic'), ('tomato', 'cheese'), ('garlic', 'cheese')], [('onion', 'chicken'), ('onion', 'tomato'), ('onion', 'garlic'), ('chic

[{'title': 'Garlic Chicken Frittata',
  'ingredients': '* 1 large onion, diced\n* 1 pound cooked chicken breast, diced\n* 3 cloves garlic, minced\n* 1 cup shredded cheese (such as cheddar or mozzarella)\n* 4 eggs',
  'directions': '1. Preheat oven to 375°F (190°C).\n2. In a large skillet, sauté the diced onion and minced garlic until softened.\n3. Add the cooked chicken to the skillet and stir to combine with the onion and garlic mixture.\n4. In a large bowl, whisk together the eggs and a pinch of salt.\n5. Add the cooked chicken and onion mixture to the eggs and stir to combine.\n6. Pour the egg mixture into a greased 9-inch (23cm) pie plate.\n7. Sprinkle the shredded cheese on top of the egg mixture.\n8. Bake for 30-35 minutes or until the eggs are set and the cheese is melted.\n9. Serve warm and enjoy!'},
 {'title': 'Garlic Onion Cheese Fritters',
  'ingredients': '* 1 large onion, finely chopped\n* 2 cloves of garlic, minced\n* 1 cup of grated cheese',
  'directions': '1. Preheat o

In [118]:
# 6 ingredients

ingredients_input = 'chicken, onion, garlic, cheese, tomato, salt'

ingredients_combinations = combinations_of_two(ingredients_input)
print('ingredients_combinations: \n', ingredients_combinations)
df_comb = data_query(ingredients_combinations)
print('\n df_comb: \n', df_comb)
ingredients_list = muse_comb(df_comb)
print('\n ingredients_list: \n', ingredients_list)
recipe_list = recipe_generator(ingredients_list)
recipe_list

ingredients_combinations: 
 [('onion', 'salt'), ('onion', 'chicken'), ('onion', 'tomato'), ('onion', 'garlic'), ('onion', 'cheese'), ('salt', 'chicken'), ('salt', 'tomato'), ('salt', 'garlic'), ('salt', 'cheese'), ('chicken', 'tomato'), ('chicken', 'garlic'), ('chicken', 'cheese'), ('tomato', 'garlic'), ('tomato', 'cheese'), ('garlic', 'cheese'), [('onion', 'salt'), ('onion', 'chicken'), ('salt', 'chicken')], [('onion', 'salt'), ('onion', 'tomato'), ('salt', 'tomato')], [('onion', 'salt'), ('onion', 'garlic'), ('salt', 'garlic')], [('onion', 'salt'), ('onion', 'cheese'), ('salt', 'cheese')], [('onion', 'chicken'), ('onion', 'tomato'), ('chicken', 'tomato')], [('onion', 'chicken'), ('onion', 'garlic'), ('chicken', 'garlic')], [('onion', 'chicken'), ('onion', 'cheese'), ('chicken', 'cheese')], [('onion', 'tomato'), ('onion', 'garlic'), ('tomato', 'garlic')], [('onion', 'tomato'), ('onion', 'cheese'), ('tomato', 'cheese')], [('onion', 'garlic'), ('onion', 'cheese'), ('garlic', 'cheese')],

[{'title': 'Chicken and Cheese Fritter',
  'ingredients': '* 1/2 cup diced chicken\n* 1 medium onion, diced\n* 2 cloves of garlic, minced\n* 1/4 cup grated cheese\n* Salt to taste',
  'directions': '1. Preheat a pan with a small amount of oil over medium heat.\n2. Add the diced chicken and cook until browned.\n3. Add the diced onion and minced garlic and cook until the onion is translucent.\n4. Mix in the grated cheese and season with salt to taste.\n5. Use a spoon to drop small amounts of the mixture into the pan and flatten slightly.\n6. Cook for an additional 2-3 minutes or until the fritter is golden brown.\n7. Serve hot and enjoy!'},
 {'title': 'Chicken Stir-Fry',
  'ingredients': '* 1 onion, sliced\n* 1/2 teaspoon salt\n* 2 cloves garlic, minced\n* 1 chicken breast, cut into bite-sized pieces',
  'directions': '1. Heat a pan over medium heat.\n2. Add the sliced onion and cook until softened.\n3. Add the minced garlic and cook for 1 minute.\n4. Add the chicken pieces and sprinkle 

'Chicken and Cheese Fritter'

# get_scores (new function that used to be random code in app.py)
Output format is OK so the code will run, but...

Issues:
- Scores seem to not differ much for both good recipes and nonsense recipes?

In [98]:
def get_scores(recipe_list):
    '''
    Generates the score of each recipe.
    
    Inputs (1):  
    recipe_list (from: recipe_generator) = a list of 3 dictionaries with 3 keys each: 'title', 'ingredients', 'directions', containing info for the 3 recipes
    
    Outputs (1):
    scores (to: final_recipes) = a list of 3 integers, containing the scores for each recipe.
    '''

    scores = []
    recipe_direction = []

    for recipe in recipe_list:
        if 'directions' in recipe:
            recipe_direction.append(recipe['directions'])
        else:
            recipe_direction.append("")

    for direction in recipe_direction:
        scores.append(model.predict_proba([direction])[0][1])

    return scores

### Testing get_scores

In [99]:
# good recipe

recipe_list = [{'title': 'Garlic Chicken Tomato Skillet',
  'ingredients': '* 1 lb chicken breast or thighs, cut into 1-inch pieces\n* 2 large tomatoes, diced\n* 3 cloves garlic, minced',
  'directions': '1. Heat a large skillet over medium-high heat.\n2. Add chicken to skillet and cook until browned, about 5-6 minutes. Remove from skillet and set aside.\n3. Add garlic to skillet and cook until fragrant, about 1 minute.\n4. Add diced tomatoes to skillet and cook until they start to soften, about 2-3 minutes.\n5. Add chicken back to skillet and stir to combine with tomato and garlic mixture. Cook until chicken is cooked through and tomato mixture is tender.'},
 {'title': 'Simple Grilled Chicken',
  'ingredients': '- 1 chicken breast\n- 1/2 onion, sliced\n- Salt, to taste',
  'directions': '1. Preheat grill to medium-high heat.\n2. Season chicken with salt, then brush both sides with a little water.\n3. Grill chicken for 6-7 minutes per side, or until cooked through.\n4. Meanwhile, grill onion slices until caramelized and slightly charred.\n5. Serve chicken with grilled onion.'},
 {'title': 'Caramelized Onion Relish',
  'ingredients': '* 1 large onion, thinly sliced\n* 1/2 teaspoon salt\n* 2 cloves garlic, minced',
  'directions': '1. In a large saucepan, combine the sliced onion, salt, and garlic. Cook over medium-low heat, stirring occasionally, until the onions are caramelized and golden brown, about 30-40 minutes.'}]

scores = get_scores(recipe_list)
scores

[0.5270266515329382, 0.5028459487913037, 0.515308160025934]

In [100]:
# nonsense recipe

recipe_list = [{'title': "Gurum's Mysterious Delight",
  'ingredients': '* Rugby grass (1 cup, chopped fresh)\n* Gum (1 tablespoon, dissolved)\n* Rum (2 tablespoons)\n* Mug (1, made from ceramic)',
  'directions': '1. Combine Rugby grass, gum, and rum in the mug.\n2. Stir clockwise for 7 times, then counterclockwise for 3 times.\n3. Serve immediately.'},
 {'title': 'Recchio Croco',
  'ingredients': '* RO Crumbs\n* Curing Oil\n* Ounces of Cream',
  'directions': '1. Crumble RO Crumbs into a crock\n2. Coat Croco with Curing Oil\n3. Ooze Cream over Croco'},
 {'title': 'Title:',
  'ingredients': '* 1 cup cherry tomatoes, halved\n* 1/2 cup mashed chickpeas\n* 1/4 cup chopped fresh cucumbers\n* 2 tablespoons olive oil\n* 2 tablespoons hummus\n* 1/2 teaspoon paprika',
  'directions': '1. Preheat oven to 375°F.\n2. In a bowl, mix together hummus, paprika, and olive oil.\n3. Dip each cherry tomato half into the hummus mixture, coating evenly.\n4. Place coated cherry tomato halves on a baking sheet.\n5. Sprinkle chopped cucumbers on top of tomato halves.\n6. Bake for 20-25 minutes, or until crispy and golden brown.\n7. Serve and enjoy!'}]

scores = get_scores(recipe_list)
scores

[0.5040546079674728, 0.5040292302849937, 0.5123256028930298]

### Testing get_scores - all the way through

In [101]:
# 3 ingredients

ingredients_input = 'chicken, onion, garlic'

ingredients_combinations = combinations_of_two(ingredients_input)
print('ingredients_combinations: \n', ingredients_combinations)
df_comb = data_query(ingredients_combinations)
print('\n df_comb: \n', df_comb)
ingredients_list = muse_comb(df_comb)
print('\n ingredients_list: \n', ingredients_list)
recipe_list = recipe_generator(ingredients_list)
print('\n recipe_list: \n', recipe_list)
scores = get_scores(recipe_list)
scores

ingredients_combinations: 
 [('onion', 'garlic'), ('onion', 'chicken'), ('garlic', 'chicken'), [('onion', 'garlic'), ('onion', 'chicken'), ('garlic', 'chicken')]]

 df_comb: 
                                          Combination  \
0                                    (onion, garlic)   
1                                   (onion, chicken)   
2                                  (garlic, chicken)   
3  [(onion, garlic), (onion, chicken), (garlic, c...   

                                               Score  
0                               [1.4235409132992736]  
1                               [1.3248796627107513]  
2                               [1.1800811339314616]  
3  [1.4235409132992736, 1.3248796627107513, 1.180...  

 ingredients_list: 
 [['onion', 'garlic', 'chicken'], ['n', 'a', 'o', 'g'], ['n', 'h', 'c', 'o']]

 recipe_list: 
 [{'title': 'Grilled Chicken with Garlic and Onion', 'ingredients': '* 1 lb boneless, skinless chicken breast\n* 1 medium onion, sliced\n* 3 cloves of ga

[0.5008613203219323, 0.5148936517953472, 0.5432702470134967]

In [102]:
# 4 ingredients

ingredients_input = 'chicken, onion, garlic, cheese'

ingredients_combinations = combinations_of_two(ingredients_input)
print('ingredients_combinations: \n', ingredients_combinations)
df_comb = data_query(ingredients_combinations)
print('\n df_comb: \n', df_comb)
ingredients_list = muse_comb(df_comb)
print('\n ingredients_list: \n', ingredients_list)
recipe_list = recipe_generator(ingredients_list)
print('\n recipe_list: \n', recipe_list)
scores = get_scores(recipe_list)
scores

ingredients_combinations: 
 [('onion', 'cheese'), ('onion', 'garlic'), ('onion', 'chicken'), ('cheese', 'garlic'), ('cheese', 'chicken'), ('garlic', 'chicken'), [('onion', 'cheese'), ('onion', 'garlic'), ('cheese', 'garlic')], [('onion', 'cheese'), ('onion', 'chicken'), ('cheese', 'chicken')], [('onion', 'garlic'), ('onion', 'chicken'), ('garlic', 'chicken')], [('cheese', 'garlic'), ('cheese', 'chicken'), ('garlic', 'chicken')], [('onion', 'cheese'), ('onion', 'garlic'), ('onion', 'chicken'), ('cheese', 'garlic'), ('cheese', 'chicken'), ('garlic', 'chicken')]]

 df_comb: 
                                           Combination  \
0                                     (onion, cheese)   
1                                     (onion, garlic)   
2                                    (onion, chicken)   
3                                    (cheese, garlic)   
4                                   (cheese, chicken)   
5                                   (garlic, chicken)   
6   [(onion, cheese),

[0.48743242692571515, 0.5074002238313483, 0.5314360536161777]

In [103]:
# 5 ingredients

ingredients_input = 'chicken, onion, garlic, cheese, tomato'

ingredients_combinations = combinations_of_two(ingredients_input)
print('ingredients_combinations: \n', ingredients_combinations)
df_comb = data_query(ingredients_combinations)
print('\n df_comb: \n', df_comb)
ingredients_list = muse_comb(df_comb)
print('\n ingredients_list: \n', ingredients_list)
recipe_list = recipe_generator(ingredients_list)
print('\n recipe_list: \n', recipe_list)
scores = get_scores(recipe_list)
scores

ingredients_combinations: 
 [('onion', 'chicken'), ('onion', 'tomato'), ('onion', 'garlic'), ('onion', 'cheese'), ('chicken', 'tomato'), ('chicken', 'garlic'), ('chicken', 'cheese'), ('tomato', 'garlic'), ('tomato', 'cheese'), ('garlic', 'cheese'), [('onion', 'chicken'), ('onion', 'tomato'), ('chicken', 'tomato')], [('onion', 'chicken'), ('onion', 'garlic'), ('chicken', 'garlic')], [('onion', 'chicken'), ('onion', 'cheese'), ('chicken', 'cheese')], [('onion', 'tomato'), ('onion', 'garlic'), ('tomato', 'garlic')], [('onion', 'tomato'), ('onion', 'cheese'), ('tomato', 'cheese')], [('onion', 'garlic'), ('onion', 'cheese'), ('garlic', 'cheese')], [('chicken', 'tomato'), ('chicken', 'garlic'), ('tomato', 'garlic')], [('chicken', 'tomato'), ('chicken', 'cheese'), ('tomato', 'cheese')], [('chicken', 'garlic'), ('chicken', 'cheese'), ('garlic', 'cheese')], [('tomato', 'garlic'), ('tomato', 'cheese'), ('garlic', 'cheese')], [('onion', 'chicken'), ('onion', 'tomato'), ('onion', 'garlic'), ('chic

[0.494125910177954, 0.517117653687935, 0.4765640054328713]

In [104]:
# 6 ingredients

ingredients_input = 'chicken, onion, garlic, cheese, tomato, salt'

ingredients_combinations = combinations_of_two(ingredients_input)
print('ingredients_combinations: \n', ingredients_combinations)
df_comb = data_query(ingredients_combinations)
print('\n df_comb: \n', df_comb)
ingredients_list = muse_comb(df_comb)
print('\n ingredients_list: \n', ingredients_list)
recipe_list = recipe_generator(ingredients_list)
print('\n recipe_list: \n', recipe_list)
scores = get_scores(recipe_list)
scores

ingredients_combinations: 
 [('onion', 'salt'), ('onion', 'chicken'), ('onion', 'tomato'), ('onion', 'garlic'), ('onion', 'cheese'), ('salt', 'chicken'), ('salt', 'tomato'), ('salt', 'garlic'), ('salt', 'cheese'), ('chicken', 'tomato'), ('chicken', 'garlic'), ('chicken', 'cheese'), ('tomato', 'garlic'), ('tomato', 'cheese'), ('garlic', 'cheese'), [('onion', 'salt'), ('onion', 'chicken'), ('salt', 'chicken')], [('onion', 'salt'), ('onion', 'tomato'), ('salt', 'tomato')], [('onion', 'salt'), ('onion', 'garlic'), ('salt', 'garlic')], [('onion', 'salt'), ('onion', 'cheese'), ('salt', 'cheese')], [('onion', 'chicken'), ('onion', 'tomato'), ('chicken', 'tomato')], [('onion', 'chicken'), ('onion', 'garlic'), ('chicken', 'garlic')], [('onion', 'chicken'), ('onion', 'cheese'), ('chicken', 'cheese')], [('onion', 'tomato'), ('onion', 'garlic'), ('tomato', 'garlic')], [('onion', 'tomato'), ('onion', 'cheese'), ('tomato', 'cheese')], [('onion', 'garlic'), ('onion', 'cheese'), ('garlic', 'cheese')],


 recipe_list: 
 [{'title': 'Garlic Chicken Fritters', 'ingredients': '* 1 chicken breast, cut into small pieces\n* 1/2 onion, chopped\n* 2 cloves garlic, minced\n* 1/4 cup grated cheese\n* 1/4 teaspoon salt', 'directions': '1. In a bowl, mix together chicken, onion, garlic, and salt.\n2. Divide the mixture into 6-8 portions and shape each portion into a ball.\n3. Flatten each ball slightly into a disk shape.\n4. Place a small amount of cheese in the center of each disk.\n5. Fold the dough over the cheese to form a ball and press edges to seal.\n6. Fry the fritters in hot oil until golden brown.'}, {'title': 'Roasted Chicken', 'ingredients': '* 1 whole chicken\n* 1 onion, peeled and chopped\n* 2 cloves of garlic, minced\n* 1 tsp salt', 'directions': '1. Preheat oven to 425°F (220°C).\n2. Rinse the chicken and pat dry with paper towels.\n3. Stuff the cavity with the chopped onion and minced garlic.\n4. Season the outside of the chicken with salt.\n5. Roast the chicken in the preheated o

[0.49166425850478057, 0.4730264070950016, 0.5210986824915466]

# thresh_regen

<br>Issues:
- Output format
    - output is 1 dict with 3 titles, 1 ingredients, 1 directions
    - dict values are in list format

In [105]:
# Most recent version from 5/14 debugging session

def thresh_regen(recipe_list, scores, model):  ###<=== Function for evaluating if the score passes the threshold and regenerating if it doesn't

    """
    This evaluates whether the score of a recipe passes or fails the threshold.
    If the recipe doesn't meet the threshold after 3 attempts, the last generated recipe is added.
    NOTE FOR FRONT-END: it's important to make sure that the outputs of the new recipe generator are the same as the
                        old version for this function to still work.
                        optimized_gptrecipe() and scoring_model() must be replaced with the actual functions

    UPDATES
    5/13/2024 by everyone:
    - Input format switched to recipe_list (previously: recipe_list was switched to lists of titles, ingredients, directions by code in app.py) 
    
    Inputs (3):
    recipe_list (from: recipe_generator) = a list of 3 dictionaries with 3 keys each: 'title', 'ingredients', 'directions', containing info for the 3 recipes
    scores (from: get_scores) = list of 3 integers, containing scores for each recipe
    model (from: get_model in app.py) = object

    Outputs (1):
    final_recipes = a list of 3 dictionaries with 3 keys each: 'title', 'ingredients', 'directions', containing info for 3 final recipes

    """

    final_recipes = {"Title": [], "Ingredients": [], "Directions": []}
    threshold = 0.3

    for i in range(len(recipe_list)):
        if scores[i] >= threshold:
            final_recipes["Title"].append(recipe_list[i]['title'])
            final_recipes["Ingredients"].append(recipe_list[i]['ingredients'])
            final_recipes["Directions"].append(recipe_list[i]['directions'])
        else:
            n = 0
            tmp_recipe = {
                "Title":recipe_list[i]['title'],
                "Ingredients":recipe_list[i]['ingredients'],
                "Directions":recipe_list[i]['directions']
                         }
            last_recipe = {"title":[],
                           "ingredients":[],
                           "directions":[]
                         }
            while n < 3:
                new_recipe = recipe_generator([tmp_recipe["Ingredients"]]) ###<=== insert actual recipe generator
                new_score = model.predict_proba([new_recipe[0]['directions']]) ###<=== insert the actual scoring model function here
                if new_score[0][1] >= threshold:
                    final_recipes["Title"].append(new_recipe[0]["title"])
                    final_recipes["Ingredients"].append(new_recipe[0]["ingredients"])
                    final_recipes["Directions"].append(new_recipe[0]["directions"])
                    break  # Exit loop if the new recipe passes the threshold
                else:
                    last_recipe = new_recipe  # Update tmp_recipe with the new recipe if the threshold isn't met
                    n += 1
            else: # Add the last generated recipe if the loop completes without finding a passing recipe
                final_recipes["Title"].append(last_recipe[0]["title"][0][0])
                final_recipes["Ingredients"].append(last_recipe[0]["ingredients"][0][0])
                final_recipes["Directions"].append(last_recipe[0]["directions"][0][0])
                break  # Exit the outer loop to prevent an unending loop

    return final_recipes

### Testing final_recipes

In [106]:
# good recipes, actual score

recipe_list = [{'title': 'Garlic Chicken Tomato Skillet',
  'ingredients': '* 1 lb chicken breast or thighs, cut into 1-inch pieces\n* 2 large tomatoes, diced\n* 3 cloves garlic, minced',
  'directions': '1. Heat a large skillet over medium-high heat.\n2. Add chicken to skillet and cook until browned, about 5-6 minutes. Remove from skillet and set aside.\n3. Add garlic to skillet and cook until fragrant, about 1 minute.\n4. Add diced tomatoes to skillet and cook until they start to soften, about 2-3 minutes.\n5. Add chicken back to skillet and stir to combine with tomato and garlic mixture. Cook until chicken is cooked through and tomato mixture is tender.'},
 {'title': 'Simple Grilled Chicken',
  'ingredients': '- 1 chicken breast\n- 1/2 onion, sliced\n- Salt, to taste',
  'directions': '1. Preheat grill to medium-high heat.\n2. Season chicken with salt, then brush both sides with a little water.\n3. Grill chicken for 6-7 minutes per side, or until cooked through.\n4. Meanwhile, grill onion slices until caramelized and slightly charred.\n5. Serve chicken with grilled onion.'},
 {'title': 'Caramelized Onion Relish',
  'ingredients': '* 1 large onion, thinly sliced\n* 1/2 teaspoon salt\n* 2 cloves garlic, minced',
  'directions': '1. In a large saucepan, combine the sliced onion, salt, and garlic. Cook over medium-low heat, stirring occasionally, until the onions are caramelized and golden brown, about 30-40 minutes.'}]

scores = [0.5270266515329382, 0.5028459487913037, 0.515308160025934]

def get_model():
    with open("model.pickle", "rb") as f:
        model = pickle.load(f)
    return model
model = get_model()

thresh_regen(recipe_list, scores, model)

{'Title': ['Garlic Chicken Tomato Skillet',
  'Simple Grilled Chicken',
  'Caramelized Onion Relish'],
 'Ingredients': ['* 1 lb chicken breast or thighs, cut into 1-inch pieces\n* 2 large tomatoes, diced\n* 3 cloves garlic, minced',
  '- 1 chicken breast\n- 1/2 onion, sliced\n- Salt, to taste',
  '* 1 large onion, thinly sliced\n* 1/2 teaspoon salt\n* 2 cloves garlic, minced'],
 'Directions': ['1. Heat a large skillet over medium-high heat.\n2. Add chicken to skillet and cook until browned, about 5-6 minutes. Remove from skillet and set aside.\n3. Add garlic to skillet and cook until fragrant, about 1 minute.\n4. Add diced tomatoes to skillet and cook until they start to soften, about 2-3 minutes.\n5. Add chicken back to skillet and stir to combine with tomato and garlic mixture. Cook until chicken is cooked through and tomato mixture is tender.',
  '1. Preheat grill to medium-high heat.\n2. Season chicken with salt, then brush both sides with a little water.\n3. Grill chicken for 6-7 

In [107]:
# nonsense recipe, actual score

recipe_list = [{'title': "Gurum's Mysterious Delight",
  'ingredients': '* Rugby grass (1 cup, chopped fresh)\n* Gum (1 tablespoon, dissolved)\n* Rum (2 tablespoons)\n* Mug (1, made from ceramic)',
  'directions': '1. Combine Rugby grass, gum, and rum in the mug.\n2. Stir clockwise for 7 times, then counterclockwise for 3 times.\n3. Serve immediately.'},
 {'title': 'Recchio Croco',
  'ingredients': '* RO Crumbs\n* Curing Oil\n* Ounces of Cream',
  'directions': '1. Crumble RO Crumbs into a crock\n2. Coat Croco with Curing Oil\n3. Ooze Cream over Croco'},
 {'title': 'Title:',
  'ingredients': '* 1 cup cherry tomatoes, halved\n* 1/2 cup mashed chickpeas\n* 1/4 cup chopped fresh cucumbers\n* 2 tablespoons olive oil\n* 2 tablespoons hummus\n* 1/2 teaspoon paprika',
  'directions': '1. Preheat oven to 375°F.\n2. In a bowl, mix together hummus, paprika, and olive oil.\n3. Dip each cherry tomato half into the hummus mixture, coating evenly.\n4. Place coated cherry tomato halves on a baking sheet.\n5. Sprinkle chopped cucumbers on top of tomato halves.\n6. Bake for 20-25 minutes, or until crispy and golden brown.\n7. Serve and enjoy!'}]

scores = [0.5040546079674728, 0.5040292302849937, 0.5123256028930298]

def get_model():
    with open("model.pickle", "rb") as f:
        model = pickle.load(f)
    return model
model = get_model()

thresh_regen(recipe_list, scores, model)

{'Title': ["Gurum's Mysterious Delight", 'Recchio Croco', 'Title:'],
 'Ingredients': ['* Rugby grass (1 cup, chopped fresh)\n* Gum (1 tablespoon, dissolved)\n* Rum (2 tablespoons)\n* Mug (1, made from ceramic)',
  '* RO Crumbs\n* Curing Oil\n* Ounces of Cream',
  '* 1 cup cherry tomatoes, halved\n* 1/2 cup mashed chickpeas\n* 1/4 cup chopped fresh cucumbers\n* 2 tablespoons olive oil\n* 2 tablespoons hummus\n* 1/2 teaspoon paprika'],
 'Directions': ['1. Combine Rugby grass, gum, and rum in the mug.\n2. Stir clockwise for 7 times, then counterclockwise for 3 times.\n3. Serve immediately.',
  '1. Crumble RO Crumbs into a crock\n2. Coat Croco with Curing Oil\n3. Ooze Cream over Croco',
  '1. Preheat oven to 375°F.\n2. In a bowl, mix together hummus, paprika, and olive oil.\n3. Dip each cherry tomato half into the hummus mixture, coating evenly.\n4. Place coated cherry tomato halves on a baking sheet.\n5. Sprinkle chopped cucumbers on top of tomato halves.\n6. Bake for 20-25 minutes, or un

In [108]:
# nonsense recipes with high scores

recipe_list = [{'title': "Gurum's Mysterious Delight",
  'ingredients': '* Rugby grass (1 cup, chopped fresh)\n* Gum (1 tablespoon, dissolved)\n* Rum (2 tablespoons)\n* Mug (1, made from ceramic)',
  'directions': '1. Combine Rugby grass, gum, and rum in the mug.\n2. Stir clockwise for 7 times, then counterclockwise for 3 times.\n3. Serve immediately.'},
 {'title': 'Recchio Croco',
  'ingredients': '* RO Crumbs\n* Curing Oil\n* Ounces of Cream',
  'directions': '1. Crumble RO Crumbs into a crock\n2. Coat Croco with Curing Oil\n3. Ooze Cream over Croco'},
 {'title': 'Title:',
  'ingredients': '* 1 cup cherry tomatoes, halved\n* 1/2 cup mashed chickpeas\n* 1/4 cup chopped fresh cucumbers\n* 2 tablespoons olive oil\n* 2 tablespoons hummus\n* 1/2 teaspoon paprika',
  'directions': '1. Preheat oven to 375°F.\n2. In a bowl, mix together hummus, paprika, and olive oil.\n3. Dip each cherry tomato half into the hummus mixture, coating evenly.\n4. Place coated cherry tomato halves on a baking sheet.\n5. Sprinkle chopped cucumbers on top of tomato halves.\n6. Bake for 20-25 minutes, or until crispy and golden brown.\n7. Serve and enjoy!'}]

scores = [0.95040546079674728, 0.95040292302849937, 0.95123256028930298]

def get_model():
    with open("model.pickle", "rb") as f:
        model = pickle.load(f)
    return model
model = get_model()

thresh_regen(recipe_list, scores, model)

{'Title': ["Gurum's Mysterious Delight", 'Recchio Croco', 'Title:'],
 'Ingredients': ['* Rugby grass (1 cup, chopped fresh)\n* Gum (1 tablespoon, dissolved)\n* Rum (2 tablespoons)\n* Mug (1, made from ceramic)',
  '* RO Crumbs\n* Curing Oil\n* Ounces of Cream',
  '* 1 cup cherry tomatoes, halved\n* 1/2 cup mashed chickpeas\n* 1/4 cup chopped fresh cucumbers\n* 2 tablespoons olive oil\n* 2 tablespoons hummus\n* 1/2 teaspoon paprika'],
 'Directions': ['1. Combine Rugby grass, gum, and rum in the mug.\n2. Stir clockwise for 7 times, then counterclockwise for 3 times.\n3. Serve immediately.',
  '1. Crumble RO Crumbs into a crock\n2. Coat Croco with Curing Oil\n3. Ooze Cream over Croco',
  '1. Preheat oven to 375°F.\n2. In a bowl, mix together hummus, paprika, and olive oil.\n3. Dip each cherry tomato half into the hummus mixture, coating evenly.\n4. Place coated cherry tomato halves on a baking sheet.\n5. Sprinkle chopped cucumbers on top of tomato halves.\n6. Bake for 20-25 minutes, or un

In [109]:
# nonsense recipes with low scores

recipe_list = [{'title': "Gurum's Mysterious Delight",
  'ingredients': '* Rugby grass (1 cup, chopped fresh)\n* Gum (1 tablespoon, dissolved)\n* Rum (2 tablespoons)\n* Mug (1, made from ceramic)',
  'directions': '1. Combine Rugby grass, gum, and rum in the mug.\n2. Stir clockwise for 7 times, then counterclockwise for 3 times.\n3. Serve immediately.'},
 {'title': 'Recchio Croco',
  'ingredients': '* RO Crumbs\n* Curing Oil\n* Ounces of Cream',
  'directions': '1. Crumble RO Crumbs into a crock\n2. Coat Croco with Curing Oil\n3. Ooze Cream over Croco'},
 {'title': 'Title:',
  'ingredients': '* 1 cup cherry tomatoes, halved\n* 1/2 cup mashed chickpeas\n* 1/4 cup chopped fresh cucumbers\n* 2 tablespoons olive oil\n* 2 tablespoons hummus\n* 1/2 teaspoon paprika',
  'directions': '1. Preheat oven to 375°F.\n2. In a bowl, mix together hummus, paprika, and olive oil.\n3. Dip each cherry tomato half into the hummus mixture, coating evenly.\n4. Place coated cherry tomato halves on a baking sheet.\n5. Sprinkle chopped cucumbers on top of tomato halves.\n6. Bake for 20-25 minutes, or until crispy and golden brown.\n7. Serve and enjoy!'}]

scores = [0.15040546079674728, 0.15040292302849937, 0.15123256028930298]

def get_model():
    with open("model.pickle", "rb") as f:
        model = pickle.load(f)
    return model
model = get_model()

thresh_regen(recipe_list, scores, model)

{'Title': ['Rugby Pitch Punch',
  'Crunchy Creamy Crostini',
  'Roasted Chickpea and Veggie Salad'],
 'Ingredients': ['* 1 cup chopped fresh Rugby grass\n* 1 tablespoon dissolved Gum\n* 2 tablespoons Rum\n* 1 ceramic Mug',
  '* * RO Crumbs\n* Curing Oil\n* Ounces of Cream',
  '* 1 cup cherry tomatoes, halved\n* 1/2 cup mashed chickpeas\n* 1/4 cup chopped fresh cucumbers\n* 2 tablespoons olive oil\n* 2 tablespoons hummus\n* 1/2 teaspoon paprika'],
 'Directions': ['1. In the ceramic Mug, mix together chopped fresh Rugby grass and dissolved Gum.\n2. Add the 2 tablespoons of Rum and stir well.\n3. Serve immediately, or let it steep for a few minutes to allow the flavors to meld.',
  '1. Preheat oven to 350°F (180°C).\n2. Spread a layer of Cream on a baking sheet.\n3. Sprinkle * RO Crumbs evenly over the Cream.\n4. Drizzle Curing Oil over the * RO Crumbs.\n5. Bake for 10-12 minutes or until the top is golden brown.\n6. Serve warm and enjoy!',
  '1. In a large bowl, combine cherry tomatoes, 

### Testing final_recipes all the way through

In [110]:
# 6 ingredients

ingredients_input = 'chicken, onion, garlic, cheese, tomato, salt'

ingredients_combinations = combinations_of_two(ingredients_input)
print('ingredients_combinations: \n', ingredients_combinations)

df_comb = data_query(ingredients_combinations)
print('\n df_comb: \n', df_comb)

ingredients_list = muse_comb(df_comb)
print('\n ingredients_list: \n', ingredients_list)

recipe_list = recipe_generator(ingredients_list)
print('\n recipe_list: \n', recipe_list)

scores = get_scores(recipe_list)
print('\n scores:', scores)

final_recipes = thresh_regen(recipe_list, scores, model)
print(final_recipes['Title'])
print(final_recipes['Ingredients'])
print(final_recipes['Directions'])

get_key_lists(final_recipes)

ingredients_combinations: 
 [('onion', 'salt'), ('onion', 'chicken'), ('onion', 'tomato'), ('onion', 'garlic'), ('onion', 'cheese'), ('salt', 'chicken'), ('salt', 'tomato'), ('salt', 'garlic'), ('salt', 'cheese'), ('chicken', 'tomato'), ('chicken', 'garlic'), ('chicken', 'cheese'), ('tomato', 'garlic'), ('tomato', 'cheese'), ('garlic', 'cheese'), [('onion', 'salt'), ('onion', 'chicken'), ('salt', 'chicken')], [('onion', 'salt'), ('onion', 'tomato'), ('salt', 'tomato')], [('onion', 'salt'), ('onion', 'garlic'), ('salt', 'garlic')], [('onion', 'salt'), ('onion', 'cheese'), ('salt', 'cheese')], [('onion', 'chicken'), ('onion', 'tomato'), ('chicken', 'tomato')], [('onion', 'chicken'), ('onion', 'garlic'), ('chicken', 'garlic')], [('onion', 'chicken'), ('onion', 'cheese'), ('chicken', 'cheese')], [('onion', 'tomato'), ('onion', 'garlic'), ('tomato', 'garlic')], [('onion', 'tomato'), ('onion', 'cheese'), ('tomato', 'cheese')], [('onion', 'garlic'), ('onion', 'cheese'), ('garlic', 'cheese')],


 recipe_list: 
 [{'title': 'Chicken and Cheese Fritters', 'ingredients': '* 1 onion, finely chopped\n* 1 chicken breast, cut into small pieces\n* 1/2 cup grated cheese\n* 2 cloves garlic, minced\n* Salt to taste', 'directions': '1. In a bowl, combine chopped onion, chicken pieces, and minced garlic.\n2. Mix well and season with salt to taste.\n3. Divide the mixture into small portions and shape into patties.\n4. Coat each patty with grated cheese.\n5. Fry the patties in a pan with oil until golden brown and crispy.\n6. Serve hot and enjoy!'}, {'title': 'Roasted Chicken with Garlic and Onion', 'ingredients': '* 1 whole chicken\n* 2 medium onions, peeled and chopped\n* 3 cloves of garlic, minced\n* 1 teaspoon salt', 'directions': '1. Preheat oven to 425°F (220°C).\n2. Season the chicken with salt.\n3. Arrange the chopped onions and minced garlic around the chicken.\n4. Roast in the preheated oven for 45-50 minutes or until the chicken is cooked through.'}, {'title': 'Cheesy Onion Fritte

# to_dictionary

In [115]:
def get_key_lists(final_recipes):
    directions, titles, ingredients = [], [], []
    titles.append(final_recipes["Title"])
    ingredients.append(final_recipes["Ingredients"])
    directions.append(final_recipes["Directions"])
    return titles, ingredients, directions

In [117]:
to_dictionary(recipe_list)

TypeError: list indices must be integers or slices, not str

# image_generator


In [ ]:
def image_generator(recipe):
    '''
    

    '''
    client = Client("ByteDance/SDXL-Lightning")
    result = client.predict(
            recipe, # str  in 'Enter your prompt (English)' Textbox component
            "1-Step",   # Literal['1-Step', '2-Step', '4-Step', '8-Step']  in 'Select inference steps' Dropdown component
            api_name="/generate_image_1"
    )
    file_path = result.split('gradio')[1]
    print(file_path)
    url = 'https://bytedance-sdxl-lightning.hf.space/file=/tmp/gradio' + file_path
    return url

In [ ]:
image_generator('garlic chicken')

# image_generator loop in app.py


In [ ]:
img_list = []
for title in titles:
    if title != None:
        img = image_generator(title)
        img_list.append(img)

# app.py stuff

### for final_recipe

### for recipe_generator

In [ ]:


directions, titles, ingredients = [], [], []
titles.append(final_recipe["Title"])
ingredients.append(final_recipe["Ingredients"])
directions.append(final_recipe["Directions"])
print(titles)
print(ingredients)
print(directions)

#### Random stuff that may or may not be useful

In [ ]:
# Example user inputs

'chicken, mustard, coconut, orange, gravy'


In [ ]:
# Example ingredients combinations

[('chicken', 'mustard'),
 ('chicken', 'coconut'),
 ('chicken', 'orange'),
 ('chicken', 'gravy'),
 ('mustard', 'coconut'),
 ('mustard', 'orange'),
 ('mustard', 'gravy'),
 ('coconut', 'orange'),
 ('coconut', 'gravy'),
 ('orange', 'gravy')]

In [ ]:
# Most recent version from 5/14 debugging session

def final_recipes(recipe_list, scores, model):  ###<=== Function for evaluating if the score passes the threshold and regenerating if it doesn't

    """
    This evaluates whether the score of a recipe passes or fails the threshold.
    If the recipe doesn't meet the threshold after 3 attempts, the last generated recipe is added.
    NOTE FOR FRONT-END: it's important to make sure that the outputs of the new recipe generator are the same as the
                        old version for this function to still work.
                        optimized_gptrecipe() and scoring_model() must be replaced with the actual functions

    UPDATES
    5/13/2024 by everyone:
    - Input format switched to recipe_list (previously: recipe_list was switched to lists of titles, ingredients, directions by code in app.py) 
    
    Inputs (3):
    recipe_list (from: recipe_generator) = a list of 3 dictionaries with 3 keys each: 'title', 'ingredients', 'directions', containing info for the 3 recipes
    scores (from: get_scores) = list of 3 integers, containing scores for each recipe
    model (from: get_model in app.py) = object

    Outputs (1):
    final_recipes = a list of 3 dictionaries with 3 keys each: 'title', 'ingredients', 'directions', containing info for 3 final recipes

    """

    final_recipes = {"Title": [], "Ingredients": [], "Directions": []}
    threshold = 0.3

    for i in range(len(recipe_list)):
        if scores[i] >= threshold:
            final_recipes["Title"].append(recipe_list[i]['title'])
            final_recipes["Ingredients"].append(recipe_list[i]['ingredients'])
            final_recipes["Directions"].append(recipe_list[i]['directions'])
        else:
            n = 0
            tmp_recipe = {
                "Title":recipe_list[i]['title'],
                "Ingredients":recipe_list[i]['ingredients'],
                "Directions":recipe_list[i]['directions']
                         }
            last_recipe = {"title":[],
                           "ingredients":[],
                           "directions":[]
                         }
            while n < 3:
                new_recipe = recipe_generator([tmp_recipe["Ingredients"]]) ###<=== insert actual recipe generator
                new_score = model.predict_proba([new_recipe[0]['directions']]) ###<=== insert the actual scoring model function here
                if new_score[0][1] >= threshold:
                    final_recipes["Title"].append(new_recipe[0]["title"])
                    final_recipes["Ingredients"].append(new_recipe[0]["ingredients"])
                    final_recipes["Directions"].append(new_recipe[0]["directions"])
                    break  # Exit loop if the new recipe passes the threshold
                else:
                    last_recipe = new_recipe  # Update tmp_recipe with the new recipe if the threshold isn't met
                    n += 1
            else: # Add the last generated recipe if the loop completes without finding a passing recipe
                final_recipes["Title"].append(last_recipe[0]["title"][0][0])
                final_recipes["Ingredients"].append(last_recipe[0]["ingredients"][0][0])
                final_recipes["Directions"].append(last_recipe[0]["directions"][0][0])
                break  # Exit the outer loop to prevent an unending loop

    return final_recipes

In [ ]:
# # previous final_recipes where code in app.py changed input to lists of titles, ingredients, and directions first

# def final_recipes(recipe_dict, scores, model):  ###<=== Function for evaluating if the score passes the threshold and regenerating if it doesn't
#     """
#     This evaluates whether the score of a recipe passes or fails the threshold.
#     If the recipe doesn't meet the threshold after 3 attempts, the last generated recipe is added.
#     NOTE FOR FRONT-END: it's important to make sure that the outputs of the new recipe generator are the same as the
#                         old version for this function to still work.
#                         optimized_gptrecipe() and scoring_model() must be replaced with the actual functions
    
#     Inputs (3): 
#     recipe_list (from: recipe_generator) = a list of 3 dictionaries with 3 keys each: 'title', 'ingredients', 'directions', containing info for the 3 recipes
#     scores (from: get_scores) = list of 3 integers, containing scores for each recipe
#     model (from: get_model in app.py) = object 
    
#     Outputs (1): 
#     final_recipes = a list of 3 dictionaries with 3 keys each: 'title', 'ingredients', 'directions', containing info for 3 final recipes
    
#     """
#     final_recipes = {"Title": [], "Ingredients": [], "Directions": []}
#     threshold = 0.5

#     for i in range(len(recipe_dict)):
#         if scores[i] >= threshold:
#             final_recipes["Title"].append(recipe_dict[i]['title'])
#             final_recipes["Ingredients"].append(recipe_dict[i]['ingredients'])
#             final_recipes["Directions"].append(recipe_dict[i]['directions'])
#         else:
#             n = 0
#             tmp_recipe = {
#                 "Title":recipe_dict[i]['title'],
#                 "Ingredients":recipe_dict[i]['ingredients'],
#                 "Directions":recipe_dict[i]['directions']
#                          }
#             last_recipe = {"title":[],
#                            "ingredients":[],
#                            "directions":[]
#                          }
#             while n < 3:
#                 new_recipe = recipe_generator([tmp_recipe["Ingredients"]]) ###<=== insert actual recipe generator
#                 new_score = model.predict_proba([new_recipe[0]['directions']]) ###<=== insert the actual scoring model function here
#                 if new_score[0][1] >= threshold:
#                     final_recipes["Title"].append(new_recipe[0]["title"])
#                     final_recipes["Ingredients"].append(new_recipe[0]["ingredients"])
#                     final_recipes["Directions"].append(new_recipe[0]["directions"])
#                     break  # Exit loop if the new recipe passes the threshold
#                 else:
#                     last_recipe = new_recipe  # Update tmp_recipe with the new recipe if the threshold isn't met
#                     n += 1
#             else: # Add the last generated recipe if the loop completes without finding a passing recipe
#                 final_recipes["Title"].append(last_recipe["title"][0][0])
#                 final_recipes["Ingredients"].append(last_recipe["ingredients"][0][0])
#                 final_recipes["Directions"].append(last_recipe["directions"][0][0])
#                 break  # Exit the outer loop to prevent an unending loop

#     return final_recipes